## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Robertsport,LR,6.7533,-11.3686,79.20,79,100,5.17,Clouds
1,1,Lavrentiya,RU,65.5833,-171.0000,29.82,97,100,2.59,Snow
2,2,Hambantota,LK,6.1241,81.1185,82.71,83,100,19.84,Clouds
3,3,Harrismith,ZA,-28.2728,29.1295,44.60,88,92,4.41,Clouds
4,4,Northam,GB,51.0333,-4.2167,60.66,71,84,10.13,Clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))

max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Robertsport,LR,6.7533,-11.3686,79.20,79,100,5.17,Clouds
2,2,Hambantota,LK,6.1241,81.1185,82.71,83,100,19.84,Clouds
5,5,Zhuhai,CN,22.2769,113.5678,76.78,78,40,9.22,Clouds
6,6,Mahebourg,MU,-20.4081,57.7000,75.49,83,40,5.75,Clouds
7,7,Butaritari,KI,3.0707,172.7902,81.27,81,29,16.67,Clouds
22,22,Forecariah,GN,9.4306,-13.0881,85.08,69,100,9.13,Clouds
35,35,Bali,ID,-8.5000,115.0000,79.77,90,88,4.00,Clouds
37,37,Moa,CU,20.6569,-74.9403,86.77,56,4,2.64,Clear
44,44,Bathsheba,BB,13.2167,-59.5167,86.61,70,20,16.11,Clouds
48,48,Buala,SB,-8.1450,159.5921,79.97,86,97,8.05,Rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                228
City                   228
Country                227
Lat                    228
Lng                    228
Max Temp               228
Humidity               228
Cloudiness             228
Wind Speed             228
Current Description    228
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df.count()

City_ID                227
City                   227
Country                227
Lat                    227
Lng                    227
Max Temp               227
Humidity               227
Cloudiness             227
Wind Speed             227
Current Description    227
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"

hotel_df["Hotel Name"] = ""

hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Robertsport,LR,79.20,Clouds,6.7533,-11.3686,
2,Hambantota,LK,82.71,Clouds,6.1241,81.1185,
5,Zhuhai,CN,76.78,Clouds,22.2769,113.5678,
6,Mahebourg,MU,75.49,Clouds,-20.4081,57.7000,
7,Butaritari,KI,81.27,Clouds,3.0707,172.7902,
22,Forecariah,GN,85.08,Clouds,9.4306,-13.0881,
35,Bali,ID,79.77,Clouds,-8.5000,115.0000,
37,Moa,CU,86.77,Clear,20.6569,-74.9403,
44,Bathsheba,BB,86.61,Clouds,13.2167,-59.5167,
48,Buala,SB,79.97,Rain,-8.1450,159.5921,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    
    "radius": 5000,
    
    "type": "lodging",
    
    "key": g_key
    
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

clean_hotel_df = hotel_df.dropna()

clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Robertsport,LR,79.20,Clouds,6.7533,-11.3686,Kwepunha Retreat
2,Hambantota,LK,82.71,Clouds,6.1241,81.1185,Bungalow 63
5,Zhuhai,CN,76.78,Clouds,22.2769,113.5678,珠海粤财假日酒店
6,Mahebourg,MU,75.49,Clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
7,Butaritari,KI,81.27,Clouds,3.0707,172.7902,Isles Sunset Lodge
22,Forecariah,GN,85.08,Clouds,9.4306,-13.0881,"Motel "" La Maison Blanche"""
35,Bali,ID,79.77,Clouds,-8.5000,115.0000,Pondok Pitaya
37,Moa,CU,86.77,Clear,20.6569,-74.9403,Carmen
44,Bathsheba,BB,86.61,Clouds,13.2167,-59.5167,Atlantis Hotel
48,Buala,SB,79.97,Rain,-8.1450,159.5921,Maringe Lagoon Lodge


In [13]:
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """

<dl>

<dt>Hotel Name</dt><dd>{Hotel Name}</dd>

<dt>City</dt><dd>{City}</dd>

<dt>Country</dt><dd>{Country}</dd>

<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>

</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map.

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))